# Load Libraries

In [3]:
import os
import pandas as pd
import numpy as np
import json

# Classes

## Segment

In [4]:
class Segment:
    # All parameters are by Month
    def __init__(self,
                 id: str,
                 lon: float, lat: float,
                 areaKm: float,
                 avgTickets: float,
                 avgCustomers: float,
                 avgPackages: float,
                 avgPackagesBySales: float,
                 packages: float,
                 sales: float,
                 customers: int,
                 avgDropSize: float,
                 setSatelitesCoverage: list[str],
                 costServedFromDC: int, # este se debe calcular con ARCE (***)
                 ):
        self.id             =   id
        self.geographyLocation = (lon, lat)
        self.areaKm         =   areaKm
        self.avgTickets     =   avgTickets
        self.avgCustomers   =   avgCustomers
        self.avgPackages    =   avgPackages
        self.avgPackagesBySales =   avgPackagesBySales
        self.packages       = packages
        self.customers      = customers
        self.sales          = sales
        self.avgDropSize    = avgDropSize
        self.setSateliteCoverage = setSatelitesCoverage
        self.costServedFromDC = costServedFromDC

## Satelite

In [30]:
class Satelite:
    def __init__(self,
                 id: str,
                 lon: float, lat: float,
                 distanceFromDC: float,
                 durationFromDC: float,
                 durationInTrafficFromDC: float,
                 capacity: dict[str,int],
                 numberVehiclesAvailable : dict[str,int],
                 costFixed: dict[str,int],
                 setSegmentCoverage: list[str],
                 ):
        self.id = id
        self.geographyLocation          = (lon, lat)
        self.distanceFromDC             = distanceFromDC
        self.durationFromDC             = durationFromDC
        self.durationInTrafficFromDC    = durationInTrafficFromDC
        self.capacity                   = capacity
        self.numberVehiclesAvailable    = numberVehiclesAvailable
        self.costFixed                  = costFixed
        self.setSegmentCoverage         = setSegmentCoverage

# Model

## Deterministic

In [ ]:
class Model:

    def __init__(self, arce: dict[(str,str),any], nameModel = "Deterministic"):
        self.model = gb.Model(nameModel)
        # variables
        self.X = {}
        self.Y = {}
        self.W = {}

        #param arce
        self.arce = arce

        #objetive & metrics
        self.results = {}
        self.metrics = {}

    def build(self, segments: list[Segment], satelites: list[Satelite]):
        self.model.reset()
        # variables
        self.__addVariables(segments, satelites)
        # objective
        self.__addObjetive(segments, satelites)
        # constraints
        self.__addConstr_LocationSatelite()
        self.__addConstr_CapacitySatelite()
        self.__addConstr_DemandSatified()

    def __addVariables(self, segments: list[Segment], satelites: list[Satelite]):
        self.Y = dict(
            [((s.id,q), self.model.addVar(vtype=GRB.BINARY, name="Y_s%s_q%s" %(s.id,q))) for s in satelites for q in s.capacity.keys()]
        )
        self.X = dict(
            [((s.id,k.id), self.model.addVar(vtype=GRB.BINARY, name="X_s%s_k%s" %(s.id,k.id))) for s in satelites for k in segments]
        )
        self.W  = dict(
            [((k.id), self.model.addVar(vtype=GRB.BINARY, name="W_k%s" % (k.id))) for k in segments]
        )

    def __addObjective(self, segments: list[Segment], satelites: list[Satelite]):
        costLocation = quicksum(
            [s.costFixed[q]*self.Y[(s.id,q)] for s in satelites for q in s.capacity.keys()]
        )
        cost_2e = quicksum(
            [self.arce[(s_id,k.id)]["totalCost"]*self.X[(s_id,k.id)] for k in segments for s_id in k.setSateliteCoverage]
        )
        cost_1e = quicksum(
            [k.costServedFromDC*self.W[(k.id)] for k in segments]
        )
        costTotal = cost_1e + cost_2e + costLocation

        self.model.setObjective(costTotal, GRB.MINIMIZE)

    def __addConstr_LocationSatelite(self, satelites: list[Satelite]):
        for s in satelites:
            nameConstraint = "R_Open_s"+str(s.id)
            self.model.addConstr(
                quicksum([self.Y[(s.id,q)] for q in s.capacity.keys()]) <= 1
                , name=nameConstraint
            )

    def __addConstr_CapacitySatelite(self, satelites: list[Satelite]):
        for s in satelites:
            nameConstraint = "R_capacity_"+str(s.id)
            self.model.addConstr(
                quicksum(
                    [self.arce[(s.id,k_id)]['fleetSize']*self.X[(s.id,k_id)] for k_id in s.setSegmentCoverage]
                )
                - quicksum(
                    [s.numberVehiclesAvailable[q]*self.Y[(s.id,q)] for q in s.capacity.keys()]
                )
                <= 0
                , name=nameConstraint
            )

    def __addConstr_DemandSatified(self, segments: list[Segment]):
        for k in segments:
            nameConstraint = "R_Demand_k"+str(k.id)
            self.model.addConstr(
                quicksum(
                    [self.X[(s_id,k.id)] for s_id in k.setSateliteCoverage].append(self.W[(k.id)])
                )
                == 1
                , name=nameConstraint
            )

    def optimizeModel(self) -> str:
        self.model.optimize()
        return self.model.Status

    def showModel(self):
        self.model.display()

    def setParams(self, params: dict[str,int]):
        for key, item in params.items():
            self.model.setParam(key,item)

# Functions

## Load Distances and Durations Matrix (Satelite to Segment)

In [44]:
def load_distances_duration_matrix(DEBUG = False) -> dict[str,dict]:
    df = pd.read_csv('../others/Levantamiento de Información/Informacion Satelites a Hexagonos.csv')
    size = len(df)
    distance = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,"distance.value"]/1000) for i in range(size)]
    )
    duration = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,'duration.value']/3600) for i in range(size)]
    )
    durationInTraffic = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,'duration_in_traffic.value']/3600) for i in range(size)]
    )
    matrixes = {
        "duration": duration,
        "distance": distance,
        "durationInTraffic": durationInTraffic
    }
    if DEBUG:
        print("-"*50)
        print("Count - distance: ", size)
        print("Count - duration: ", size)
        print("Count - duration in traffic: ", size)
    return matrixes

## Load segments

In [33]:
def load_segments(DEBUG = False) -> list[Segment]:
    segments = []
    df = pd.read_csv('../others/Levantamiento de Información/Caracterización de hexagonos de La Paz.csv')
    df.fillna(0, inplace=True)
    for i in range(len(df)):
        setSatelitesCoverage = ["empty"]
        costServedFromDC = -1
        newSegment = Segment(str(df.h3_address[i]),
                             df.loc[i,'lon.centroid'],
                             df.loc[i,'lat.centroid'],
                             df.loc[i,'area.km2'],
                             df.loc[i,'media.boletas.mes'],
                             df.loc[i,'media.clientes.mes'],
                             df.loc[i,'media.suma.cajas.mes'],
                             df.loc[i,'media.cajas.por.venta'],
                             df.loc[i,'suma.cajas'],
                             df.loc[i,'Cantidad.ventas'],
                             df.loc[i,'Total.clientes'],
                             df.loc[i,'media.cajas.cliente'],
                             setSatelitesCoverage,
                             costServedFromDC,
                             )
        segments.append(newSegment)
    if DEBUG:
        print("-"*50)
        print("Count of SEGMENTS: ", len(segments))
        print("First Segment:")
        print(json.dumps(segments[0].__dict__,indent=2,default=str))
    return segments

## Load satelites

In [34]:
def load_satelites(DEBUG = False) ->list[Satelite]:
    satelites = []
    df = pd.read_csv('../others/Levantamiento de Información/Informacion CD a Satelites.csv')
    df.fillna(0, inplace=True)
    for i in range(len(df)):
        capacity = {}
        numberVehiclesAvailable = {}
        costFixed = {}
        setSegmentCoverage = ['empty']
        newSatelite = Satelite(str(df.nombre[i]),
                               df.longitud[i],
                               df.latitud[i],
                               df.loc[i,'distance.value']/1000,
                               df.loc[i,'duration.value']/3600,
                               df.loc[i,'duration_in_traffic.value']/3600,
                               capacity,
                               numberVehiclesAvailable,
                               costFixed,
                               setSegmentCoverage,
                               )
        satelites.append(newSatelite)
    if DEBUG:
        print("-"*50)
        print("Count of SATELITES: ", len(satelites))
        print("First Satelite:")
        print(json.dumps(satelites[0].__dict__,indent=2,default=str))
    return satelites

# Arce

# MAIN

## Deterministic

In [36]:
df = load_segments(True)

--------------------------------------------------
Count of SEGMENTS:  881
First Segment:
{
  "id": "89b3218a69bffff",
  "geographyLocation": [
    -68.0308444046886,
    -16.577683655286
  ],
  "areaKm": 0.111550134969712,
  "avgTickets": 0.0,
  "avgCustomers": 0.0,
  "avgPackages": 0.0,
  "avgPackagesBySales": 0.0,
  "packages": 0.0,
  "customers": 0.0,
  "sales": 0.0,
  "avgDropSize": 0.0,
  "setSateliteCoverage": [
    "empty"
  ],
  "costServedFromDC": -1
}


In [37]:
df = load_satelites(True)

--------------------------------------------------
Count of SATELITES:  9
First Satelite:
{
  "id": "Abaroa",
  "geographyLocation": [
    -68.128476,
    -16.512483
  ],
  "distanceFromDC": 11.117,
  "durationFromDC": 0.5772222222222222,
  "durationInTrafficFromDC": 0.6044444444444445,
  "capacity": {},
  "numberVehiclesAvailable": {},
  "costFixed": {},
  "setSegmentCoverage": [
    "empty"
  ]
}


In [45]:
matrix = load_distances_duration_matrix(True)

--------------------------------------------------
Count - distance:  7929
Count - duration:  7929
Count - duration in traffic:  7929


In [46]:
881*9

7929